In [1]:
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")


CUDA available: True
CUDA version: 12.1
GPU Device: NVIDIA GeForce RTX 4090


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
wandb_api_key = os.getenv("WANDB_API_KEY")

if not huggingface_api_key:
    raise ValueError("HUGGINGFACE_API_KEY not set")

if not wandb_api_key:
    raise ValueError("WANDB_API_KEY not set")

In [3]:
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)

from peft import (
    LoraConfig,
    get_peft_model
)

import wandb
import pandas as pd
from datasets import Dataset

In [4]:
from huggingface_hub import login

login(token=huggingface_api_key)

In [5]:
wandb.login(key=wandb_api_key)

run = wandb.init(
    project="repurposed-llm-phishing-classifier",
    job_type="train",
    anonymous="allow"
)

wandb: Currently logged in as: morganj-lee01 (morganj-lee01-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\morga\_netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [6]:
base_model_path = "../models/llama_models/llama-3.2-3B"
new_model_path = "../models/tuned_models/llama-3.2-3B-phishing-classifier-seq-cls"
train_dataset_path = "../processed_data/train.csv"
test_dataset_path = "../processed_data/test.csv"

In [7]:
train_df = pd.read_csv(train_dataset_path)

train_df.head()

,system,user,assistant
0,You are a classification system designed to ca...,Message for review: write me back please year ...,True
1,You are a classification system designed to ca...,Message for review: I just picked up Razor SDK...,False
2,You are a classification system designed to ca...,"Message for review: vacation goodmorning , i w...",False
3,You are a classification system designed to ca...,"Message for review: On Thu, Aug 08, 2002 at 11...",False
4,You are a classification system designed to ca...,Message for review: wellheads shoreline has se...,False


In [8]:
model_config = {
    "torch_dtype": torch.bfloat16,
    "attn_implementation": "flash_attention_2",
    "device_map": "auto"
}

In [9]:
# WARNING: flash_attention_2 required pip install flash-attn which needs C++ builds tools
# It also takes absolutely forever to compile because it's compiling CUDA kernels

model = AutoModelForSequenceClassification.from_pretrained(
    base_model_path,
    num_labels=2,
    device_map=model_config["device_map"],
    torch_dtype=model_config["torch_dtype"],
    attn_implementation=model_config["attn_implementation"]
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ../models/llama_models/llama-3.2-3B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

In [11]:
tokenizer.chat_template = None

tokenizer.chat_template = """<|begin_of_text|>{% for message in messages %}<|start_header_id|>{{ message['role'] }}<|end_header_id|>{{ message['content'] }}<|eot_id|>{% endfor %}"""


In [12]:
test_messages = [
    {"role": "system", "content": "You are a classification system..."},
    {"role": "user", "content": "Message for review: test"},
    {"role": "assistant", "content": "true"}
]

output = tokenizer.apply_chat_template(test_messages, tokenize=False)
print("Length:", len(output))
print("Content:", output)

Length: 236
Content: <|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a classification system...<|eot_id|><|start_header_id|>user<|end_header_id|>Message for review: test<|eot_id|><|start_header_id|>assistant<|end_header_id|>true<|eot_id|>


In [13]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

In [14]:
from functools import partial

train_dataset = Dataset.from_pandas(pd.read_csv(train_dataset_path))

def format_chat_template_batch(examples, tokenizer):
    formatted_texts = []
    labels = []

    for system, user, assistant in zip(
        examples["system"],
        examples["user"],
        examples["assistant"]
    ):
        row_json = [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
            {"role": "assistant", "content": ""}
        ]
        formatted_texts.append(tokenizer.apply_chat_template(row_json, tokenize=False))
        labels.append(1 if str(assistant).lower() == "true" else 0)

    tokenized = tokenizer(
        formatted_texts,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors=None
    )

    examples.update(tokenized)
    examples["labels"] = labels
    return examples


format_with_tokenizer = partial(format_chat_template_batch, tokenizer=tokenizer)

train_dataset = train_dataset.map(
    format_with_tokenizer,
    batched=True,
    batch_size=100
)

Map:   0%|          | 0/14920 [00:00<?, ? examples/s]

In [15]:
test_dataset = Dataset.from_pandas(pd.read_csv(test_dataset_path))

test_dataset = test_dataset.map(
    format_with_tokenizer,
    batched=True,
    batch_size=100
)

Map:   0%|          | 0/3730 [00:00<?, ? examples/s]

In [16]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "down_proj", "up_proj"
    ],
    bias="none",
    task_type="SEQ_CLS",
    inference_mode=False
)

model = get_peft_model(model, peft_config)

In [17]:
training_arguments = TrainingArguments(
    output_dir=new_model_path,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    optim="adamw_torch_fused",
    num_train_epochs=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    do_eval=True,
    warmup_steps=100,
    learning_rate=1e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="wandb",
    neftune_noise_alpha=0.1,
    run_name="actual_run_2",
    logging_first_step=True,
    logging_dir="../logs",
    logging_strategy="steps",
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=2,
)

In [18]:
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2,
    early_stopping_threshold=0.003
)

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    callbacks=[early_stopping_callback]
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.009100,0.054648
2,0.056900,0.057878
3,0.000000,0.076753


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=1401, training_loss=0.07599274610447765, metrics={'train_runtime': 3956.9468, 'train_samples_per_second': 37.706, 'train_steps_per_second': 1.178, 'total_flos': 3.902364682341581e+17, 'train_loss': 0.07599274610447765, 'epoch': 3.0})

In [22]:
wandb.finish()

eval/loss,▁▂█
eval/runtime,▂▁█
eval/samples_per_second,▇█▁
eval/steps_per_second,▇█▁
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█
train/grad_norm,▃█▄▂▅▁▁▂▂▂▁▁▁▂▂▂▂▂▂▁▁▂▃▂▁▂▁▁▁▁▂▁▁▁▂▁▁▁▁▁
train/learning_rate,▁▃█████████████▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆
train/loss,█▁▁▁▃▁▃▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
eval/loss,0.07675
eval/runtime,134.1741


In [23]:
trainer.model.save_pretrained(new_model_path)